In [2]:
!pip install requests 

In [ ]:
yanolja/EEVE-Korean-10.8B-v1.0

In [ ]:
import requests
import random
import gradio as gr

# 감정 정규화 리스트
allowed_emotions = {
    "기쁨": ["기쁨", "행복", "좋음", "신남", "즐거움", "만족", "기분 좋음", "기쁜", "환호", "신난", "희열", "웃김", "유쾌"],
    "슬픔": ["슬픔", "우울", "쓸쓸함", "눈물", "외로움", "허무함", "애통", "고독", "마음이 아프다", "비통"],
    "분노": ["분노", "화남", "짜증", "열받음", "격분", "폭발", "분개", "화가 나", "극대노", "불쾌"],
    "걱정": ["걱정", "불안", "초조", "긴장", "염려", "근심", "조바심", "심란", "불안정", "걱정돼"],
    "감동": ["감동", "고마움", "감사", "훈훈함", "감격", "경이", "가슴 뭉클", "감탄", "놀라움", "깊은 인상"],
    "당황": ["당황", "어리둥절", "멘붕", "놀람", "놀라움", "어색", "혼란", "예상 못함"],
    "혐오": ["혐오", "역겨움", "싫음", "불쾌함", "기분 나쁨", "거부감", "징그러움"],
    "무관심": ["무관심", "심심함", "지루함", "무감정", "냉담", "흥미 없음", "무표정"]
}

model_questions = [
    ("오늘 회사에서 발표를 망쳐서 너무 우울했어.", "슬픔"),
    ("시험 결과가 잘 나와서 기분이 좋아!", "기쁨"),
    ("친구가 나 몰래 파티를 준비했더라, 감동이었어.", "감동"),
    ("갑자기 연락이 끊긴 친구가 너무 걱정돼.", "걱정"),
    ("지하철에서 누가 밀었는데 사과도 안 하더라. 열받았어.", "분노"),
    ("오늘 하루종일 비 와서 괜히 쓸쓸했어.", "슬픔"),
    ("갑자기 편지를 받았는데 너무 고마웠어.", "감동"),
    ("친구가 나를 기쁘게 해줬어.", "기쁨"),
    ("갑자기 깜짝 놀랐어.", "당황"),
    ("상사가 나를 인정해줘서 뿌듯했어.", "기쁨"),
    ("새로운 사람을 만나서 당황했어.", "당황")
]

def normalize_emotion(emotion):
    for key, synonyms in allowed_emotions.items():
        if emotion.strip() in synonyms:
            return key
    return "기타"

def generate_model_question():
    emotion = random.choice(list(allowed_emotions.keys()))
    # 문장의 길이를 모델 질문과 비슷하게 설정
    sentence_length = random.randint(10, 20)  # 대략적인 문장 길이 범위 (단어 수)
    
    # 감정을 지정한 후, 그 감정에 맞는 문장을 모델에게 요청
    prompt = f"""다음 감정에 맞는 문장을 생성해 주세요. 문장은 {sentence_length}자 정도로 작성해 주세요.
감정: {emotion}"""
    
    payload = {
        "model": "EEVE-Korean-10.8B",
        "prompt": prompt,
        "stream": False
    }
    
    try:
        response = requests.post("http://localhost:11434/api/generate", json=payload)
        if response.status_code == 200:
            generated_sentence = response.json()["response"].strip()
            return generated_sentence, emotion
        else:
            return f"ERROR: {response.status_code}", emotion
    except Exception as e:
        return f"요청 실패: {e}", emotion

def user_turn(user_sentence, user_emotion, turn, score_user, score_eeve):
    user_emotion = normalize_emotion(user_emotion)
    predicted = get_emotion_from_eeve(user_sentence)
    if predicted == user_emotion:
        score_eeve += 1
        result = f"✅ EEVE 정답! ({predicted})"
    else:
        result = f"❌ EEVE 오답. EEVE의 판단은 {predicted}"

    turn += 1
    return "", "", result, "", f"👤 User: {score_user}점 | 🤖 EEVE: {score_eeve}점", turn, score_user, score_eeve, "▶️ EEVE의 턴입니다! 감정을 맞혀보세요."

def model_turn(user_guess, model_sentence, correct_emotion, turn, score_user, score_eeve):
    user_guess = normalize_emotion(user_guess)
    if user_guess == correct_emotion:
        score_user += 1
        result = f"✅ 정답! ({correct_emotion})"
    else:
        result = f"❌ 오답! 정답은 {correct_emotion}"

    turn += 1
    return "", "", result, "", f"👤 User: {score_user}점 | 🤖 EEVE: {score_eeve}점", turn, score_user, score_eeve, "▶️ 당신의 턴입니다! 문장을 입력하고 감정을 선택하세요."

def end_game(score_user, score_eeve):
    if score_user > score_eeve:
        message = f"🎉 축하합니다! 당신이 승리했습니다. (User: {score_user}점, EEVE: {score_eeve}점)"
    elif score_user < score_eeve:
        message = f"💻 EEVE가 승리했습니다. (User: {score_user}점, EEVE: {score_eeve}점)"
    else:
        message = f"🙌 무승부입니다! (User: {score_user}점, EEVE: {score_eeve}점)"
    return message, "🛑 게임이 종료되었습니다."

def reset_game():
    return 0, 0, "🎮 게임 진행 중입니다.", "", "👤 User: 0점 | 🤖 EEVE: 0점", "▶️ 당신의 턴입니다! 문장을 입력하고 감정을 선택하세요."

with gr.Blocks(title="공감력 배틀") as demo:
    gr.Markdown("## 🎮 공감력 배틀 - EEVE vs USER", elem_id="main_title")

    turn = gr.State(0)
    score_user = gr.State(0)
    score_eeve = gr.State(0)
    model_q = gr.State("")
    model_q_emotion = gr.State("")

    turn_display = gr.Markdown("▶️ **당신의 턴입니다! 문장을 입력하고 감정을 선택하세요.**", elem_id="turn_display")
    score_display = gr.Markdown("👤 User: 0점 | 🤖 EEVE: 0점")

    start_game_btn = gr.Button("게임 시작")
    game_status = gr.Markdown("🕹️ 대기 중...", elem_id="game_status")

    with gr.Row() as user_input_row:
        user_sentence = gr.Textbox(label="당신이 낸 문장", max_lines=1, scale=3)
        user_emotion = gr.Dropdown(label="감정", choices=list(allowed_emotions.keys()), allow_custom_value=True, scale=1)
        user_submit = gr.Button("맞춰봐라 EEVE!", scale=1, elem_id="user_submit")

    result_output = gr.Markdown("""


**결과가 여기에 표시됩니다.**
""", elem_id="result_output")

    with gr.Row() as model_input_row:
        model_sentence = gr.Textbox(label="EEVE가 낸 문장", max_lines=1, scale=3)
        user_guess = gr.Dropdown(label="이 문장의 감정은?", choices=list(allowed_emotions.keys()), allow_custom_value=True)
        model_submit = gr.Button("정답 제출", scale=1, elem_id="model_submit")

    end_game_btn = gr.Button("게임 종료")
    end_game_output = gr.Textbox(label="게임 결과", lines=2)

    demo.css = """
    #main_title {
        text-align: center;
        font-weight: bold;
    }
    #result_output, #turn_display, #game_status, #end_game_output {
        text-align: center;
        font-weight: bold;
    }
    #user_submit, #model_submit {
        height: 100px;  /* 버튼의 높이를 100px로 설정 (기존의 두 배 크기) */
    }
    """

    start_game_btn.click(
        fn=reset_game,
        inputs=[],
        outputs=[score_user, score_eeve, game_status, model_sentence, score_display, turn_display]
    )

    user_submit.click(
        fn=user_turn,
        inputs=[user_sentence, user_emotion, turn, score_user, score_eeve],
        outputs=[user_sentence, user_emotion, result_output, model_sentence, score_display, turn, score_user, score_eeve, turn_display]
    ).then(
        fn=generate_model_question,
        inputs=[],
        outputs=[model_q, model_q_emotion]
    ).then(
        fn=lambda q: q,
        inputs=[model_q],
        outputs=[model_sentence]
    )

    model_submit.click(
        fn=model_turn,
        inputs=[user_guess, model_q, model_q_emotion, turn, score_user, score_eeve],
        outputs=[user_sentence, user_emotion, result_output, model_sentence, score_display, turn, score_user, score_eeve, turn_display]
    )

    end_game_btn.click(
        fn=end_game,
        inputs=[score_user, score_eeve],
        outputs=[end_game_output, game_status]
    )

if __name__ == "__main__":
    demo.launch(share=True)


In [ ]:
# EEVE 공감력 배틀 전체 코드
import requests
import random
import gradio as gr
import json
from pathlib import Path
from tabulate import tabulate

# 감정 정규화 리스트
allowed_emotions = {
    "기쁨": ["기쁨", "행복", "좋음", "신남", "즐거움", "만족", "기분 좋음", "기쁜", "환호", "신난", "희열", "웃김", "유쾌"],
    "슬픔": ["슬픔", "우울", "쓸쓸함", "눈물", "외로움", "허무함", "애통", "고독", "마음이 아프다", "비통"],
    "분노": ["분노", "화남", "짜증", "열받음", "격분", "폭발", "분개", "화가 나", "극대노", "불쾌"],
    "걱정": ["걱정", "불안", "초조", "긴장", "염려", "근심", "조바심", "심란", "불안정", "걱정돼"],
    "감동": ["감동", "고마움", "감사", "훈훈함", "감격", "경이", "가슴 뭉클", "감탄", "놀라움", "깊은 인상"],
    "당황": ["당황", "어리둥절", "멘붕", "놀람", "놀라움", "어색", "혼란", "예상 못함"],
    "혐오": ["혐오", "역겨움", "싫음", "불쾌함", "기분 나쁨", "거부감", "징그러움"],
    "무관심": ["무관심", "심심함", "지루함", "무감정", "냉담", "흥미 없음", "무표정"]
}

RANKING_FILE = "ranking.json"

EMOJI_MAP = {
    "기쁨": "😊", "슬픔": "😢", "분노": "😡", "걱정": "😟",
    "감동": "🥹", "당황": "😳", "혐오": "🤢", "무관심": "😐", "기타": "❓"
}

def normalize_emotion(emotion):
    for key, synonyms in allowed_emotions.items():
        if emotion.strip() in synonyms:
            return key
    return "기타"

def generate_model_question():
    emotion = random.choice(list(allowed_emotions.keys()))
    sentence_length = random.randint(10, 20)
    prompt = f"""다음 감정에 맞는 문장을 생성해 주세요. 문장은 {sentence_length}자 정도로 작성해 주세요.\n감정: {emotion}"""
    payload = {"model": "EEVE-Korean-10.8B", "prompt": prompt, "stream": False}
    try:
        response = requests.post("http://localhost:11434/api/generate", json=payload)
        if response.status_code == 200:
            return response.json()["response"].strip(), emotion
        else:
            return f"ERROR: {response.status_code}", emotion
    except Exception as e:
        return f"요청 실패: {e}", emotion

def save_score(username, score_user, score_eeve, turns):
    if Path(RANKING_FILE).exists():
        with open(RANKING_FILE, "r", encoding="utf-8") as f:
            data = json.load(f)
    else:
        data = {}
    data[username] = {
        "user_score": score_user,
        "eeve_score": score_eeve,
        "turns": turns
    }
    with open(RANKING_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def get_ranking():
    if not Path(RANKING_FILE).exists():
        return "<div style='text-align:center;'>랭킹이 아직 없습니다.</div>"
    with open(RANKING_FILE, "r", encoding="utf-8") as f:
        data = json.load(f)
    sorted_data = sorted(data.items(), key=lambda x: x[1]["user_score"], reverse=True)
    table = [["순위", "사용자", "User 점수", "EEVE 점수", "턴 수"]]
    for i, (user, info) in enumerate(sorted_data):
        table.append([
            f"{i+1}위",
            user,
            f"{info['user_score']}점",
            f"{info['eeve_score']}점",
            f"{int(info['turns']/2)}턴"
        ])
    html_table = tabulate(table, headers="firstrow", tablefmt="html")
    return f"<div style='text-align:center;'>{html_table}</div>"

def get_emotion_from_eeve(text):
    return random.choice(list(allowed_emotions.keys()))

def user_turn(user_sentence, user_emotion, turn, score_user, score_eeve):
    user_emotion = normalize_emotion(user_emotion)
    predicted = get_emotion_from_eeve(user_sentence)
    emoji = EMOJI_MAP.get(predicted, "❓")
    if predicted == user_emotion:
        score_eeve += 1
        result = f"✅ EEVE 정답! ({predicted} {emoji})"
    else:
        result = f"❌ EEVE 오답. EEVE의 판단은 {predicted} {emoji}"
    turn += 1
    return "", "", result, "", f"👤 User: {score_user}점 | 🤖 EEVE: {score_eeve}점", turn, score_user, score_eeve, "▶️ EEVE의 턴입니다! 감정을 맞혀보세요."

def model_turn(user_guess, model_sentence, correct_emotion, turn, score_user, score_eeve):
    user_guess = normalize_emotion(user_guess)
    emoji = EMOJI_MAP.get(correct_emotion, "❓")
    if user_guess == correct_emotion:
        score_user += 1
        result = f"✅ 정답! ({correct_emotion} {emoji})"
    else:
        result = f"❌ 오답! 정답은 {correct_emotion} {emoji}"
    turn += 1
    return "", "", result, "", f"👤 User: {score_user}점 | 🤖 EEVE: {score_eeve}점", turn, score_user, score_eeve, "▶️ 당신의 턴입니다! 문장을 입력하고 감정을 선택하세요."

def end_game_and_save(username, score_user, score_eeve, turn):
    save_score(username, score_user, score_eeve, turn)
    if score_user > score_eeve:
        return f"🎉 축하합니다! {username}님이 승리했습니다. (User: {score_user}점, EEVE: {score_eeve}점)", "🛑 게임이 종료되었습니다."
    elif score_user < score_eeve:
        return f"💻 EEVE가 승리했습니다. (User: {score_user}점, EEVE: {score_eeve}점)", "🛑 게임이 종료되었습니다."
    else:
        return f"🙌 무승부입니다! (User: {score_user}점, EEVE: {score_eeve}점)", "🛑 게임이 종료되었습니다."

def reset_game():
    return 0, 0, "🎮 게임 진행 중입니다.", "", "👤 User: 0점 | 🤖 EEVE: 0점", "▶️ 당신의 턴입니다! 문장을 입력하고 감정을 선택하세요."

with gr.Blocks(title="공감력 배틀") as demo:
    gr.Markdown("## 🎮 공감력 배틀 - EEVE vs USER", elem_id="main_title")

    username = gr.Textbox(label="당신의 이름을 입력하세요", placeholder="닉네임 또는 이름")

    turn = gr.State(0)
    score_user = gr.State(0)
    score_eeve = gr.State(0)
    model_q = gr.State("")
    model_q_emotion = gr.State("")

    turn_display = gr.Markdown("▶️ **당신의 턴입니다! 문장을 입력하고 감정을 선택하세요.**", elem_id="turn_display")
    score_display = gr.Markdown("👤 User: 0점 | 🤖 EEVE: 0점")

    start_game_btn = gr.Button("게임 시작")
    game_status = gr.Markdown("🕹️ 대기 중...", elem_id="game_status")

    with gr.Row():
        user_sentence = gr.Textbox(label="당신이 낸 문장", max_lines=1, scale=3)
        user_emotion = gr.Dropdown(label="감정", choices=list(allowed_emotions.keys()), allow_custom_value=True, scale=1)
        user_submit = gr.Button("맞춰봐라 EEVE!", scale=1, elem_id="user_btn")

    result_output = gr.Markdown("**결과가 여기에 표시됩니다.**", elem_id="result_output")

    with gr.Row():
        model_sentence = gr.Textbox(label="EEVE가 낸 문장", max_lines=1, scale=3)
        user_guess = gr.Dropdown(label="이 문장의 감정은?", choices=list(allowed_emotions.keys()), allow_custom_value=True)
        model_submit = gr.Button("정답 제출", elem_id="model_btn")

    end_game_btn = gr.Button("게임 종료")
    end_game_output = gr.Textbox(label="게임 결과", lines=2)

    show_rank_btn = gr.Button("📊 랭킹 보기")
    ranking_output = gr.HTML()

    demo.css = """
    #main_title {
        text-align: center;
        font-weight: bold;
    }
    #result_output, #turn_display, #game_status, #end_game_output {
        text-align: center;
        font-weight: bold;
    }
    #user_btn, #model_btn {
        height: 80px;
        font-size: 20px;
    }
    """

    start_game_btn.click(
        fn=reset_game,
        inputs=[],
        outputs=[score_user, score_eeve, game_status, model_sentence, score_display, turn_display]
    )

    user_submit.click(
        fn=user_turn,
        inputs=[user_sentence, user_emotion, turn, score_user, score_eeve],
        outputs=[user_sentence, user_emotion, result_output, model_sentence, score_display, turn, score_user, score_eeve, turn_display]
    ).then(
        fn=generate_model_question,
        inputs=[],
        outputs=[model_q, model_q_emotion]
    ).then(
        fn=lambda q: q,
        inputs=[model_q],
        outputs=[model_sentence]
    )

    model_submit.click(
        fn=model_turn,
        inputs=[user_guess, model_q, model_q_emotion, turn, score_user, score_eeve],
        outputs=[user_sentence, user_emotion, result_output, model_sentence, score_display, turn, score_user, score_eeve, turn_display]
    )

    end_game_btn.click(
        fn=end_game_and_save,
        inputs=[username, score_user, score_eeve, turn],
        outputs=[end_game_output, game_status]
    )

    show_rank_btn.click(
        fn=get_ranking,
        inputs=[],
        outputs=[ranking_output]
    )

if __name__ == "__main__":
    demo.launch(share=True)